# HHEM

Install dependencies



In [ ]:
%pip install datasets --quiet # for importing from huggingface and ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Load model

In [ ]:
from transformers import AutoModelForSequenceClassification
hhem = AutoModelForSequenceClassification.from_pretrained('vectara/hallucination_evaluation_model', trust_remote_code=True)
hhem = hhem.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

configuration_hhem_v2.py:   0%|          | 0.00/760 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vectara/hallucination_evaluation_model:
- configuration_hhem_v2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.


modeling_hhem_v2.py:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

### Halubench

In [ ]:
from datasets import load_dataset, Dataset

ds = load_dataset("PatronusAI/HaluBench")
data = ds["test"].to_pandas()

# store sources for subsets
sources = data.source_ds.unique().tolist()
sources.remove('RAGTruth')
sources.remove('halueval')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.00k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/7.51M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/14900 [00:00<?, ? examples/s]

In [ ]:
import os, json

def chunked_data(data, chunk_size=16):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

def save_to_json(filename, scores, source):
    # check if json exists
    if os.path.exists(f'{filename}.json'):
        with open(f'{filename}.json', 'r') as f:
            results_dict = json.load(f)
    else:
        results_dict = {}

    # add the results under source key
    results_dict[source] = [{'score': score} for score in scores]

    # write the updated dictionary
    with open(f'{filename}.json', 'w') as f:
        json.dump(results_dict, f, indent=2)

In [ ]:
def run_metrics(data, sources, model, filename):
    for source in sources:
        # extract subset
        subset = data[data.source_ds == source]
        results = []

        # prepare data
        test_data = list(zip((subset['passage'] + '\n' + subset['question']), subset['answer']))

        scores = []

        for chunk in chunked_data(test_data, chunk_size=1):
          # predict on the current chunk and store the results
          predictions = model.predict(chunk).tolist()
          scores.extend(predictions)

        # save results to json
        save_to_json(filename, scores, source)

        print("Completed evaluation on {0} dataset. Length of scores: {1}".format(source, len(scores)))

In [ ]:
filename = 'HHEM'

run_metrics(data, sources, hhem, filename)

Token indices sequence length is longer than the specified maximum sequence length for this model (3586 > 512). Running this sequence through the model will result in indexing errors


Completed evaluation on covidQA dataset. Length of scores: 1000


### ELI5

In [ ]:
from datasets import load_dataset, Dataset

ds = load_dataset("explodinggradients/ELI5")
data = ds["train"].to_pandas()

def run_metrics(data, model, filename):
    # extract subset
    subset = data
    results = []
    source = 'eli5'

    # prepare data
    test_data = list(zip((subset['reference'] + '\n' + subset['user_input']), subset['response']))

    scores = []

    for chunk in chunked_data(test_data, chunk_size=1):
      # predict on the current chunk and store the results
      predictions = model.predict(chunk).tolist()
      scores.extend(predictions)

    # save results to json
    save_to_json(filename, scores, source)

    print("Completed evaluation. Length of scores: {0}".format(len(scores)))
    return scores

filename = 'HHEM'

scores = run_metrics(data, hhem, filename)

Completed evaluation. Length of scores: 100
